In [ ]:
%matplotlib inline
# not sure if necessary

In [2]:
import finetune
import importlib
import torch

In [1]:
# run this cell when you change stuff in the finetune module, and don't want to restart the notebook's kernel.
importlib.reload(finetune)
finetune.structure_checkpoints()  # If this prints 'Cannot safely create ...', it means that the folder probably already exists

NameError: name 'importlib' is not defined

In [6]:
# inception model doesn't work.
# models = ['resnet', 'alexnet', 'densenet', 'squeezenet', 'vgg']
#
# for name in models:
#     print('[TRAINING]:', name)
# finetune.CLASSIFIER_NAME = name
importlib.reload(finetune)

for classifier_name in ['resnet']:
    for occlusion_name in ['0', 'GAUSSIAN', 'SOFTMAX']:
        for dataset in finetune.DATASET_OPTIONS:
            print('STARTING: model={} | dataset={} | occlusion={}'.format(classifier_name, dataset, occlusion_name))
            finetune.CLASSIFIER_NAME = classifier_name
            finetune.NUM_EPOCHS = None
            finetune.FEATURE_EXTRACT = False # default
            finetune.DATASET = dataset
            finetune.OCCLUSION_NAME = occlusion_name
            finetune.OPTIMIZER_NAME = 'adam' # default
            # finetune.print_hypers() # shows params

            # Initialize the model for this run
            model_ft = finetune.initialize_model(_verbose=False)

            model, hist, state = finetune.finetune_model(model_ft, checkpoint_save=10)

            # Save the model after the last epoch
            finetune.save_model(state)
            del model_ft
            del model
            del hist
            del state


STARTING: model=resnet | dataset=StanfordCars | occlusion=0


Epoch 9/9999 val: 100%|██████████| 503/503 [00:28<00:00, 17.66it/s, loss=1.3437, accuracy=0.63383]


[CHECKPOINT] ../../data/finetuned_models/StanfordCars/resnet/resnet_StanfordCars_E10_occ0.pth


Epoch 19/9999 val: 100%|██████████| 503/503 [00:45<00:00, 10.96it/s, loss=1.3083, accuracy=0.67786]


[CHECKPOINT] ../../data/finetuned_models/StanfordCars/resnet/resnet_StanfordCars_E20_occ0.pth


...

[CHECKPOINT] ../../data/finetuned_models/FGVC-Aircraft/resnet/resnet_FGVC-Aircraft_E40_occSOFTMAX.pth


Epoch 49/9999 val: 100%|██████████| 417/417 [00:22<00:00, 18.54it/s, loss=2.5075, accuracy=0.52955]


[CHECKPOINT] ../../data/finetuned_models/FGVC-Aircraft/resnet/resnet_FGVC-Aircraft_E50_occSOFTMAX.pth


Epoch 56/9999 val: 100%|██████████| 417/417 [00:22<00:00, 18.23it/s, loss=2.2414, accuracy=0.53285]


Training complete in 148m 51s
Best val Acc: 0.563456
[CHECKPOINT] ../../data/finetuned_models/FGVC-Aircraft/resnet/resnet_FGVC-Aircraft_E10000_occSOFTMAX.pth


In [3]:

importlib.reload(finetune)

for classifier_name in ['resnet']:
    for occlusion_name in [None]:
        for dataset in finetune.DATASET_OPTIONS:
            print('STARTING: model={} | dataset={} | occlusion={}'.format(classifier_name, dataset, occlusion_name))
            finetune.CLASSIFIER_NAME = classifier_name
            finetune.NUM_EPOCHS = None
            finetune.FEATURE_EXTRACT = False # default
            finetune.DATASET = dataset
            finetune.OCCLUSION_NAME = occlusion_name
            finetune.OPTIMIZER_NAME = 'adam' # default
            # finetune.print_hypers() # shows params

            # Initialize the model for this run
            model_ft = finetune.initialize_model(_verbose=False)

            model, hist, state = finetune.finetune_model(model_ft, checkpoint_save=10)

            # Save the model after the last epoch
            finetune.save_model(state)
            del model_ft
            del model
            del hist
            del state

STARTING: model=resnet | dataset=StanfordCars | occlusion=None


Epoch 9/9999 val: 100%|██████████| 503/503 [00:28<00:00, 17.45it/s, loss=0.73429, accuracy=0.80721]


[CHECKPOINT] ../../data/finetuned_models/StanfordCars/resnet/resnet_StanfordCars_E10.pth


Epoch 19/9999 val: 100%|██████████| 503/503 [00:28<00:00, 17.57it/s, loss=0.81922, accuracy=0.79801]


[CHECKPOINT] ../../data/finetuned_models/StanfordCars/resnet/resnet_StanfordCars_E20.pth


...

[CHECKPOINT] ../../data/finetuned_models/FGVC-Aircraft/resnet/resnet_FGVC-Aircraft_E30.pth


Epoch 39/9999 val: 100%|██████████| 417/417 [00:23<00:00, 18.01it/s, loss=1.867, accuracy=0.59616] 


[CHECKPOINT] ../../data/finetuned_models/FGVC-Aircraft/resnet/resnet_FGVC-Aircraft_E40.pth


Epoch 43/9999 val: 100%|██████████| 417/417 [00:23<00:00, 17.72it/s, loss=1.9135, accuracy=0.58206]


Training complete in 81m 13s
Best val Acc: 0.636964
[CHECKPOINT] ../../data/finetuned_models/FGVC-Aircraft/resnet/resnet_FGVC-Aircraft_E10000.pth


In [ ]:
model_save_path = finetune.format_model_path('resnet',
                           'FGVC-Aircraft',
                           30)

checkpoint = finetune.load_checkpoint(model_save_path)

train_acc_history, train_loss_history, val_acc_history = finetune.get_information_from_checkpoint(checkpoint, plot=True)

In [ ]:
optimizer_state_dict = checkpoint['optimizer_state_dict']
finetune.NUM_EPOCHS = 10
checkpoint_model = finetune.get_model_architecture('resnet')
checkpoint_model.load_state_dict(checkpoint['model_state_dict'])

test_model, hist, state = finetune.finetune_model(checkpoint_model,
                                                  'adam',
                                                  optimizer_state_dict=optimizer_state_dict,
                                                  checkpoint_save=5,
                                                  is_retrain=True)


In [ ]:
finetune.save_model(state)